# 🤖 Breadboard AI Tutor (Student Chatbot)

Welcome!

This chatbot will help you learn breadboard experiments.

### ✅ What you need to do:
1. Run the setup cell below  
2. Enter the Gemini API key (teacher will provide it)  
3. Start asking questions in the chat window  

Type **exit** anytime to stop.

In [ ]:
!pip install -q google-generativeai gradio

In [ ]:
import json
import google.generativeai as genai
from getpass import getpass

print("✅ Loading Breadboard Experiment Dataset...")

# Load dataset correctly from repo folder
with open("../data/experiments.json") as f:
    experiments = json.load(f)

print("✅ Experiments Loaded:", len(experiments))

# Secure Gemini Key input
GEMINI_API_KEY = getpass("Enter Gemini API Key: ")

genai.configure(api_key=GEMINI_API_KEY)

print("✅ Gemini AI Tutor Connected Successfully!")

In [ ]:
SYSTEM_PROMPT = """
You are a Breadboard Tutor Chatbot.

Rules:
- Answer ONLY using the dataset context provided.
- Do NOT use external knowledge.
- Always explain like a teacher for students.
- If answer is not found, say:
  "Sorry, I can only answer from the breadboard lessons."
"""

def retrieve_context(question):
    question_lower = question.lower()
    context = ""

    # Search experiments + components
    for exp in experiments:
        if any(word in exp["experiment_name"].lower() for word in question_lower.split()):
            context += f"\\nExperiment: {exp['experiment_name']}\\n"
            context += "\\n".join(exp["steps"])
            context += "\\n"

        for comp, info in exp["component_working"].items():
            if comp.lower() in question_lower:
                context += f"\\nComponent: {comp}\\nExplanation: {info}\\n"

    return context.strip()


def breadboard_tutor(question):
    context = retrieve_context(question)

    if context == "":
        return "Sorry, I can only answer from the breadboard lessons."

    model = genai.GenerativeModel("gemini-pro")

    prompt = f"""
{SYSTEM_PROMPT}

DATASET CONTEXT:
{context}

STUDENT QUESTION:
{question}

Now respond clearly with step-by-step help.
"""

    response = model.generate_content(prompt)
    return response.text

In [ ]:
import gradio as gr

print("🚀 Launching Student Chatbot Interface...")

demo = gr.ChatInterface(
    fn=breadboard_tutor,
    title="🤖 Breadboard AI Tutor",
    description="Ask questions about LED circuits, buzzers, resistors, breadboards, and troubleshooting."
)

demo.launch()